In [ ]:
import os
import pyodbc
import pandas as pd
import numpy as np
import warnings
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import figure
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

import seaborn as sns

#warnings.filterwarnings(action='once')
warnings.filterwarnings(action='ignore')

import datetime, json
import os
from google.cloud import secretmanager

In [ ]:
#pd.set_option('display.max_columns', None)

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
import os
import pyodbc
import pandas as pd

set_secrets_as_envs()

def create_tdv_conn(connection_string):
    """Funksjon for å lage connection til TDV med connection streng."""
    conn = pyodbc.connect(connection_string)
    conn.setencoding(encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_WMETADATA, encoding='utf-32le')
    return conn
    
conn = create_tdv_conn(os.environ['TDV_CONN_STR'])
sql = """select * from FAM_PP_STONAD_UR"""

df = pd.read_sql(sql, conn)

In [ ]:
df.head(3)

In [ ]:
df = df[['PERIODE','FK_PERSON1_OMSORGSPERSON', 'FK_PERSON1_PLEIETRENGENDE','RELASJON_TIL_OMSORGSPERSON', 'YRKESSTATUS', 'KOMMUNE_NR',
       'DIAGNOSE_TYPE', 'DIAGNOSE_KODE', 'IDENTDATO','VEDTAKS_DATO', 'ANVIST_FOM', 'ANVIST_TOM', 'ANVIST_DATO',
       'YTELSE_TYPE', 'BRUTTO_BEREGNINGSGRUNNLAG', 'FORBRUKTE_DAGER','FORBRUKTE_DAGER_VEKTET', 'UTTAKSGRAD', 'DAGSATS', 'DAGSATS_TOTAL',
       'BELOP', 'ANTALL_DAGER', 'UTBETALINGSGRAD', 'PLEIEBEHOV','OPPGITT_TILSYN', 'SOKERS_TAPTE_TIMER', 'SOKERS_TAPTE_ARBEIDSTID',
       'UTFALL', 'MAX_VEDTAKSDATO']]

In [ ]:
# converting FK_PERSON1_OMSORGSPERSON & FK_PERSON1_PLEIETRENGENDE to string

df['FK_PERSON1_OMSORGSPERSON'] = df['FK_PERSON1_OMSORGSPERSON'].astype(str).apply(lambda x: x.replace('.0', ''))
df['FK_PERSON1_PLEIETRENGENDE'] = df['FK_PERSON1_PLEIETRENGENDE'].astype(str).apply(lambda x: x.replace('.0', ''))

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
filt = (df.duplicated(keep=False))
df.loc[filt].sort_values(by = 'FK_PERSON1_OMSORGSPERSON')

In [ ]:
df.isna().sum().plot(kind = 'bar')

In [ ]:
 df.drop(columns = 'MAX_VEDTAKSDATO', axis = 1, inplace = True)

In [ ]:
df.isna().sum().plot(kind = 'bar')

In [ ]:
# det er en sammenheng mellom tømme YRKESSTATUS og UTBETALINGSGRAD verdier (kanskje de ikke er hentet riktig?)

filt = (df['YRKESSTATUS'].isna() & df['UTBETALINGSGRAD'].isna())
df.loc[filt]

In [ ]:
# av de 6316 er det som er - linjer 2291

filt = (df['YRKESSTATUS'].isna() & df['UTBETALINGSGRAD'].isna() & df['BELOP'].astype(str).str.contains('-'))
df.loc[filt]

#len(df[df['BELOP'].astype(str).str.contains('-')])

In [ ]:
# Kalkulerer SOKERS_TAPTE_TIMER i reall tid (hours:minutes) basert 
def finn_tid(x):
    tid = ''
    if pd.isna(x):
        tid = x
    elif 'H' not in x and 'M' not in x and 'S' not in x:
        tid = x[2:]
    elif 'M' not in x:
        tid = x[2: x.find('H')]
    elif 'H' not in x and 'M' in x:
        tid = str(float(round(int(x[2: x.find('M')])/60, 3)))
    else:
        tid = x[2: x.find('H')] + '.' + str(round(int(x[x.find('H')+1 : x.find('M')])/60, 3)).split('.')[-1]  
    return tid

In [ ]:
# Extract timer og minutter(kalkulert til timer) fra SOKERS_TAPTE_TIMER --> kanskje kan brukes til noe fornuftig
df['SOKERS_TAPTE_TIMER_TM'] = df['SOKERS_TAPTE_TIMER'].apply(finn_tid).astype(float)

In [ ]:
# se på rader hvor FORBRUKTE_DAGER er ikke lik ANTALL_DAGER

filt = (df['FORBRUKTE_DAGER'] != df['ANTALL_DAGER'])
df.loc[filt, ['FORBRUKTE_DAGER', 'ANTALL_DAGER']]

In [ ]:
filt = ((df['BRUTTO_BEREGNINGSGRUNNLAG'] > (111477*6)) & (df['PERIODE'] == 202208))
df.loc[filt, 'DAGSATS_TOTAL']

In [ ]:
2573*15/100

In [ ]:
df['DAGSATS'].max()

In [ ]:
df.columns

In [ ]:

#np.count_nonzero
df[['PERIODE', 'UTFALL', 'FK_PERSON1_PLEIETRENGENDE']].pivot_table(index = 'PERIODE', columns = 'UTFALL', aggfunc = lambda x: len(x.unique()), fill_value=0)

In [ ]:
df.columns
# grouppert på anvist (stonadsperiode)
# fam_pp_perioder_utbet_grader

In [ ]:
#filt = (df['RELASJON_TIL_OMSORGSPERSON'].isna())
#df.loc[filt]

In [ ]:
#df[['ANVIST_FOM', 'ANVIST_TOM', 'ANVIST_DATO']]

In [ ]:
#df['YTELSE_TYPE'].unique()

In [ ]:
df_begrenset = df[['PERIODE','FK_PERSON1_OMSORGSPERSON', 'FK_PERSON1_PLEIETRENGENDE',
       'RELASJON_TIL_OMSORGSPERSON','DIAGNOSE_KODE',
       'VEDTAKS_DATO', 'ANVIST_FOM', 'ANVIST_TOM', 'BRUTTO_BEREGNINGSGRUNNLAG', 'FORBRUKTE_DAGER',
       'FORBRUKTE_DAGER_VEKTET', 'UTTAKSGRAD', 'DAGSATS', 'DAGSATS_TOTAL','BELOP', 'ANTALL_DAGER', 'UTBETALINGSGRAD',
        'PLEIEBEHOV','SOKERS_TAPTE_TIMER', 'SOKERS_TAPTE_ARBEIDSTID']]

In [ ]:
df_begrenset.head()

In [ ]:
round(df_begrenset.describe().drop('PERIODE', axis = 1), 1)

In [ ]:
#df_begrenset.columns

In [ ]:
filt = (df_begrenset['PLEIEBEHOV'] == 200)
df_begrenset.loc[filt]

In [ ]:
# sjekk pleiebehov for denne FK_PERSON1_PLEIETRENGENDE

filt = (df_begrenset['FK_PERSON1_PLEIETRENGENDE'].isin(['1339862686']) & (df_begrenset['PERIODE'].isin([202111, 202112])))
df_begrenset.loc[filt].sort_values(by = 'ANVIST_FOM')

In [ ]:
filt = (df['RELASJON_TIL_OMSORGSPERSON'].isna())

df.loc[filt]

#df['RELASJON_TIL_OMSORGSPERSON'].isna().sum()

In [ ]:
# bare 1 OMSORGSPERSON men 200% pleiebehov

filt = (df_begrenset['FK_PERSON1_OMSORGSPERSON'] == '1174428389')
df_begrenset[filt]

In [ ]:
df['PERIODE_EXTRACT_F'] = df['ANVIST_FOM'].apply(lambda x: str(x).replace('-','')[:6])
df['PERIODE_EXTRACT_T'] = df['ANVIST_TOM'].apply(lambda x: str(x).replace('-','')[:6])

In [ ]:
df['PERIODE_EXTRACT_F']

In [ ]:
# sjekker at beregning av periode fra ANVIST_FOM og ANVIST_TOM er likt

filt = (df['PERIODE_EXTRACT_F'] != df['PERIODE_EXTRACT_T'])
df.loc[filt]

In [ ]:
filt = (df['PERIODE'].astype(str) != df['PERIODE_EXTRACT_T'])
df[filt]

In [ ]:
df['ANVIST_FOM'] = pd.to_datetime(df['ANVIST_FOM'])
#df['STATISTIKK_FOM'] = pd.to_datetime(df['STATISTIKK_FOM'])

In [ ]:
# antall OMSORGSPERSON per pleiebehov distiburt på PERIODE

df[['PERIODE', 'PLEIEBEHOV', 'FK_PERSON1_OMSORGSPERSON']].pivot_table(index = 'PERIODE',
                                                                                   columns = 'PLEIEBEHOV', aggfunc = lambda x: len(x.unique()), fill_value=0, margins = True)

In [ ]:
# antall PLEIETRENGENDE per pleiebehov distiburt på PERIODE

df[['PERIODE', 'PLEIEBEHOV', 'FK_PERSON1_PLEIETRENGENDE']].pivot_table(index = 'PERIODE',
                                                                                   columns = 'PLEIEBEHOV', aggfunc = lambda x: len(x.unique()), fill_value=0, margins = True)

In [ ]:
# antall PLEIETRENGENDE per pleiebehov distiburt på PERIODE

df[['PERIODE', 'BELOP']].pivot_table(index = 'PERIODE',columns = 'BELOP', aggfunc = lambda x: np.sum(x), fill_value=0, margins=True)

In [ ]:
filt = (df['PERIODE'] == 202201)
np.sum(df.loc[filt, 'BELOP'])

In [ ]:
#df_begrenset['PERIODE'].value_counts()
df.groupby(['PERIODE', 'PLEIEBEHOV'])['FK_PERSON1_OMSORGSPERSON'].nunique()

In [ ]:
#filt = ((df_begrenset['PERIODE'] == 202201) & (df_begrenset['PLEIEBEHOV'] == 200))
#len(df_begrenset.loc[filt, 'FK_PERSON1_OMSORGSPERSON'].unique())

In [ ]:
#filt = df_begrenset['PERIODE'] == 202201
#df_begrenset.loc[filt, 'FK_PERSON1_OMSORGSPERSON'].unique():

In [ ]:
df_begrenset['PERIODE_EXTRACT_F'].value_counts()
df_begrenset.groupby(['PERIODE_EXTRACT_F', 'PLEIEBEHOV'])['FK_PERSON1_OMSORGSPERSON'].nunique()

In [ ]:
df_begrenset[['PERIODE_EXTRACT_F', 'PLEIEBEHOV', 'FK_PERSON1_OMSORGSPERSON']].pivot_table(index = 'PERIODE_EXTRACT_F',
                                                                                   columns = 'PLEIEBEHOV', aggfunc = lambda x: len(x.unique()), fill_value=0)

In [ ]:
df_begrenset['DIAGNOSE_KODE'].value_counts(dropna=True)[:10].plot(kind='bar')

In [ ]:
# topp 5 diagnoser for 200% pleietrengende
filt = (df_begrenset['PLEIEBEHOV'] == 100)

df_begrenset.loc[filt,'DIAGNOSE_KODE'].value_counts(dropna=True)[:5].plot(kind='bar')

In [ ]:
# topp 5 diagnoser for 200% pleietrengende
filt = (df_begrenset['PLEIEBEHOV'] == 200)

df_begrenset.loc[filt,'DIAGNOSE_KODE'].value_counts(dropna=True)[:5].plot(kind='bar')

In [ ]:
#df_begrenset.pivot_table(index = 'FK_PERSON1_PLEIETRENGENDE', columns = 'DIAGNOSE_KODE', aggfunc='count', sort = True)

In [ ]:
pd.crosstab(df_begrenset['PERIODE'], df_begrenset['PLEIEBEHOV'])

In [ ]:
# pleiebehov er 0

filt = (df_begrenset['PLEIEBEHOV'] == 0)
df_begrenset.loc[filt].sort_values(by = 'FK_PERSON1_PLEIETRENGENDE')

In [ ]:
# 494677725 har kun 1 + linje og ikke - fra spørringen ovenfor

filt = ((df_begrenset['PLEIEBEHOV'] == 0) & (df_begrenset['FK_PERSON1_PLEIETRENGENDE'] == '494677725'))
df_begrenset.loc[filt]

In [ ]:
filt = ((df_begrenset['FK_PERSON1_OMSORGSPERSON'] == '166623817') & (df_begrenset['PERIODE'] == 202111))
df_begrenset.loc[filt, ['PERIODE', 'VEDTAKS_DATO', 'ANVIST_FOM',
       'ANVIST_TOM',  
       'BRUTTO_BEREGNINGSGRUNNLAG', 'FORBRUKTE_DAGER',
       'FORBRUKTE_DAGER_VEKTET', 'UTTAKSGRAD', 'DAGSATS', 'DAGSATS_TOTAL',
       'BELOP', 'ANTALL_DAGER', 'UTBETALINGSGRAD', 'PLEIEBEHOV',
       'SOKERS_TAPTE_TIMER', 'SOKERS_TAPTE_ARBEIDSTID']]

In [ ]:
#tid = ''
#s = 'PT3H22M30S'[2:]
#tid = s[:s.find('H')] + '.' + str(round(int(s[s.find('H')+1:s.find('M')])/60, 3)).split('.')[-1]
#float(tid)

In [ ]:
#l = ['PT7H38M', 'PT10H38M', 'PT8H38M24S', 'PT3H38M', 'PT3H38M', 'PT3H38M','PT8H38M', 'PT1H38M','PT38M', 'PT2H38M','PT5H38M']

#x = 'PT38M'
#tid = x[2: x.find('H')] + '.' + str(round(int(x[x.find('H')+1 : x.find('M')])/60, 3)).split('.')[-1]  
#finn_tid(x)
#for i in l:
#    print(i, finn_tid(i))
#tid

In [ ]:
df_begrenset['SOKERS_TAPTE_TIMER_TM'] = df_begrenset['SOKERS_TAPTE_TIMER'].apply(finn_tid).astype(float)

In [ ]:
df_begrenset['SOKERS_TAPTE_TIMER_TM'].dtype
#type(df_begrenset['SOKERS_TAPTE_TIMER_TM'][0])

In [ ]:
df_begrenset['SOKERS_TAPTE_TIMER_TM'].describe()

In [ ]:
df_begrenset['SOKERS_TAPTE_TIMER_TM']

In [ ]:
df_begrenset.head()

In [ ]:
filt = (df_begrenset['DAGSATS'] != df_begrenset['DAGSATS_TOTAL'])
df_begrenset.loc[filt]

In [ ]:
filt = (df_begrenset['FK_PERSON1_PLEIETRENGENDE'] == '1396589741')
df_begrenset.loc[filt]

In [ ]:
filt = (df_begrenset['SOKERS_TAPTE_ARBEIDSTID'] != 100.00)
df_begrenset.loc[filt]

In [ ]:
filt = (df_begrenset['FK_PERSON1_PLEIETRENGENDE'] == '1116381753')
df_begrenset.loc[filt].sort_values(by = 'ANVIST_FOM')

In [ ]:
df_begrenset['FK_PERSON1_OMSORGSPERSON'].astype(str)

In [ ]:
filt = df['UTTAKSGRAD'] < 30.00
df_begrenset.loc[filt]

In [ ]:
filt = (df_begrenset['FK_PERSON1_OMSORGSPERSON'] == '558422114')
df_begrenset.loc[filt]

In [ ]:
df_begrenset[['PERIODE', 'FK_PERSON1_OMSORGSPERSON','FORBRUKTE_DAGER', 'FORBRUKTE_DAGER_VEKTET', 'PLEIEBEHOV','UTTAKSGRAD', 'DAGSATS',
       'DAGSATS_TOTAL', 'BELOP', 'ANTALL_DAGER', 'UTBETALINGSGRAD',
        'SOKERS_TAPTE_TIMER', 'SOKERS_TAPTE_ARBEIDSTID',
       'SOKERS_TAPTE_TIMER_TM']].head()

In [ ]:
df_begrenset['UTTAKSGRAD']